# Stochastic Gradient Descent

### Data Loading

In [1]:
import pandas as pd
import numpy as np

trn_imputed_dum_X = pd.read_csv('./Dataset/trn_imputed_dum_X.csv')
tst_imputed_dum_X = pd.read_csv('./Dataset/tst_imputed_dum_X.csv')
trn_origin_Y      = pd.read_csv('./Dataset/trn_origin_Y.csv')
tst_origin_X      = pd.read_csv('./Dataset/tst_origin_X.csv')

print(trn_imputed_dum_X.shape)
print(trn_origin_Y.shape)

(79853, 17)
(79853, 1)


### 1) Hyperparameter optimization

In [4]:
#from sklearn.svm import SVC
from sklearn import linear_model
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold

param = {'alpha':[0.00001, 0.0001, 0.0005, 0.001, 0.005, 0.01]}

def clf_SGD(trn_X, trn_Y, svc_param):
    clf_SVC = linear_model.SGDClassifier()
    clf = GridSearchCV(clf_SVC, param, n_jobs=6, cv=5, scoring='roc_auc')
    clf.fit(trn_X, trn_Y)
    return clf  

### 2) Training Model

In [6]:
clf_model = clf_SGD(trn_imputed_dum_X, trn_origin_Y.values.ravel(), param)

/home/ec2-user/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)
/home/ec2-user/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)
/home/ec2-us

/home/ec2-user/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)
/home/ec2-user/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)
/home/ec2-us

In [8]:
clf_model.grid_scores_

/home/ec2-user/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_search.py:761: DeprecationWarning: The grid_scores_ attribute was deprecated in version 0.18 in favor of the more elaborate cv_results_ attribute. The grid_scores_ attribute will not be available from 0.20
  DeprecationWarning)


[mean: 0.49399, std: 0.00376, params: {'alpha': 1e-05},
 mean: 0.49399, std: 0.00376, params: {'alpha': 0.0001},
 mean: 0.49400, std: 0.00376, params: {'alpha': 0.0005},
 mean: 0.49401, std: 0.00373, params: {'alpha': 0.001},
 mean: 0.49401, std: 0.00374, params: {'alpha': 0.005},
 mean: 0.49422, std: 0.00393, params: {'alpha': 0.01}]


### 3) DO PREDICTION

In [6]:
clf_yhat = clf_model.best_estimator_.predict(tst_imputed_dum_X)
clf_prob = clf_model.best_estimator_.predict_proba(tst_imputed_dum_X)
clf_prob_renewal = clf_prob[:,1]
display(clf_prob_renewal)

AttributeError: predict_proba is not available when  probability=False

### 4) Export to output file

In [ ]:
tst_output = pd.DataFrame(tst_origin_X[['id', 'premium']])

clf_prob_df = pd.DataFrame(clf_prob_renewal.reshape(len(clf_prob_renewal),1), columns=['renewal'])
pd_incentives = pd.DataFrame(np.zeros((len(clf_prob_renewal), 1)), columns=['incentives'], dtype=np.float32)
pd_improve = pd.DataFrame(np.zeros((len(clf_prob_renewal), 1)), columns=['improvement'], dtype=np.float32)

tst_output = pd.concat([tst_out, clf_prob_df, pd_improve, pd_incentives], axis=1)
#display(tst_output)

tst_output.to_csv('./Dataset/tst_renewal_predicted_SVM.csv', index=False)